In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/resnet-v2/tensorflow2/50-feature-vector/1/saved_model.pb
/kaggle/input/resnet-v2/tensorflow2/50-feature-vector/1/variables/variables.index
/kaggle/input/resnet-v2/tensorflow2/50-feature-vector/1/variables/variables.data-00000-of-00001
/kaggle/input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Test/pigmented benign keratosis/ISIC_0024371.jpg
/kaggle/input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Test/pigmented benign keratosis/ISIC_0024358.jpg
/kaggle/input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Test/pigmented benign keratosis/ISIC_0024337.jpg
/kaggle/input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Test/pigmented benign keratosis/ISIC_0024382.jpg
/kaggle/input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Test/pigmented benign keratosis/ISIC_0024420.jpg
/

In [9]:
pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator



train_dir = '/kaggle/input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Train' 

test_dir = '/kaggle/input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Test' 



train_datagen = ImageDataGenerator(rescale=1/255.)

test_datagen = ImageDataGenerator(rescale=1/255.)



train_data = train_datagen.flow_from_directory(train_dir,

                                               target_size = (224,224),

                                               batch_size=25,

                                               class_mode ='categorical'

                                               )

test_data = test_datagen.flow_from_directory(test_dir,

                                               target_size = (224,224),

                                               batch_size=25,

                                               class_mode ='categorical'

                                               )

E0000 00:00:1732197639.195191      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:479
D1121 14:00:39.203493606      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D1121 14:00:39.203509179      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D1121 14:00:39.203513029      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D1121 14:00:39.203516013      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D1121 14:00:39.203518855      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defa

Found 2239 images belonging to 9 classes.
Found 118 images belonging to 9 classes.


In [3]:
import tensorflow_hub as hub

import tensorflow as tf

hub_layer = hub.KerasLayer("https://kaggle.com/models/google/resnet-v2/TensorFlow2/50-feature-vector/1",

                   trainable=False)



hub_layer_wrapper = tf.keras.layers.Lambda(lambda x: hub_layer(x))



m = tf.keras.Sequential([

    tf.keras.layers.InputLayer(input_shape=(224, 224, 3)),

    hub_layer_wrapper,

    tf.keras.layers.Dense(9, activation='softmax')

])

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
I0000 00:00:1732197653.742608      13 service.cc:145] XLA service 0x5d25b62677e0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732197653.742656      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1732197653.742661      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1732197653.742664      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1732197653.742666      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1732197653.742671      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1732197653.742674      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00

In [4]:
m.compile(loss='categorical_crossentropy',

                     optimizer=tf.keras.optimizers.Adam(),

                     metrics=['accuracy'])

In [5]:
from tensorflow.keras import backend as K


In [6]:
def f1_score(y_true, y_pred):

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))

    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))

    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

    precision = true_positives / (predicted_positives + K.epsilon())

    recall = true_positives / (possible_positives + K.epsilon())

    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())

    return f1_val

In [7]:
m.compile(loss='categorical_crossentropy',

                     optimizer=tf.keras.optimizers.Adam(),

                     metrics=[f1_score])

In [8]:
history = m.fit(train_data, epochs=4, validation_data=test_data)


Epoch 1/4


/usr/local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


NotFoundError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/local/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/local/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/usr/local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/usr/local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/usr/local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/local/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/local/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/local/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 545, in dispatch_qu

In [ ]:
m.evaluate(test_data)

In [ ]:
def precprocess_image(filename):

  img = tf.io.read_file(filename)

  img = tf.image.decode_image(img)

  img = tf.image.resize(img, (224, 224))

  img = img/255.

  return img

In [ ]:
img1 = precprocess_image("")

img2 =precprocess_image("/content/Dandelion.jpg")




In [ ]:
import matplotlib.pyplot as plt



# Make predictions

predictions = []

for img in [img1, img2, img3, img4, img5]:

  prediction = m.predict(tf.expand_dims(img, axis=0))  # Add batch dimension

  predicted_class = tf.argmax(prediction, axis=1).numpy()[0]

  predictions.append(predicted_class)



# Define class names

class_names = ['actinic keratosis','basal cell carcinoma','dermatofibroma','melanoma','nevus','pigmented benign keratosis','seborrheic keratosis','squamous cell carcinoma','vascular lesion']



# Plot the images with predictions

fig, axes = plt.subplots(1, 5, figsize=(15, 3))

for i, img in enumerate([img1, img2, img3, img4, img5]):

  axes[i].imshow(img)

  axes[i].set_title(f"Predicted: {class_names[predictions[i]]}")

  axes[i].axis('off')



plt.show()

In [ ]:
test_image = "/content/PranavTest.jpg"

prediction = m.predict(tf.expand_dims(img, axis=0))  # Add batch dimension

predicted_class = tf.argmax(prediction, axis=1).numpy()[0]

predictions.append(predicted_class)

In [ ]:
preprocessed_image = precprocess_image(test_image)

prediction = m.predict(tf.expand_dims(img, axis=0))  # Add batch dimension

predicted_class = tf.argmax(prediction, axis=1).numpy()[0]

predicted_class

In [ ]:
import matplotlib.pyplot as plt

import matplotlib.image as mpimg # Import the module and alias it as 'mpimg'

import numpy as np

img = precprocess_image("/content/PranavTest.jpg")

img = tf.expand_dims(img, axis=0)

pred = m.predict(img)

plt.figure(figsize=(10,10)) # Create a figure with the desired figsize

plt.subplot(5,5,i)

plt.imshow(mpimg.imread("/content/PranavTest.jpg"))

plt.title(class_names[np.argmax(pred)])

plt.axis("off")

plt.show()

**Seems we need to do data augmentation lets use GANs**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


def build_generator():
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=100))
    model.add(layers.Dense(784, activation='sigmoid'))  # Adjust based on your image shape
    model.add(layers.Reshape((28, 28, 1)))  # Adjust based on your image shape
    return model

def build_discriminator():
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28, 1)))  # Adjust based on your image shape
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Compile the GAN
generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Combine models
z = layers.Input(shape=(100,))
img = generator(z)
discriminator.trainable = False
validity = discriminator(img)
combined = models.Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer='adam')

# Step 2: Train the GAN
def train_gan(train, epochs, batch_size):
    for epoch in range(epochs):
        # Select a random batch of real images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_images = X_train[idx]

        # Generate fake images
        noise = np.random.normal(0, 1, (batch_size, 100))
        fake_images = generator.predict(noise)

        # Train the discriminator
        d_loss_real = discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))

        # Print progress
        if epoch % 100 == 0:
            print(f"{epoch} [D loss: {d_loss[0]:.4f}, acc.: {100*d_loss[1]:.2f}%] [G loss: {g_loss:.4f}]")

# Step 3: Generate Augmented Data
def generate_augmented_data(num_samples):
    noise = np.random.normal(0, 1, (num_samples, 100))
    generated_images = generator.predict(noise)
    return generated_images



# Step 4: Train the GAN
train_gan(train_data, epochs=100, batch_size=64)

# Step 5: Augment the dataset
augmented_images = generate_augmented_data(2000)  # Generate 2000 new images
X_augmented = np.concatenate((X_train, augmented_images))
y_augmented = np.concatenate((y_train, y_train[:2000]))  # Adjust labels accordingly

# Step 6: Train ResNet with Augmented Data
# Prepare the ResNet model
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(len(np.unique(y_augmented)), activation='softmax')(x)  # Adjust number of classes

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_augmented, y_augmented, epochs=10, batch_size=32, validation_split=0.2)
